# IRIS PROCESSING NOTEBOOK (IRIS with screen)

## ReadMe :

- This notebook is designed for the IRIS version WITH A SCREEN. This version outputs a .txt file with date/time information, height, and voltage.

- Make sure to enter the correct filename in the input cell

- Run every cell, the processed data will be written in a separate .txt file
<br/><br/>

Author : Paul Billecocq, GRIMP, Université de Sherbrooke, QC, Canada

Contact : paul.billecocq@usherbrooke.ca

In [1]:
import pandas as pd
import iris_core as iris

### 1. Input IRIS file name

In [2]:
iris_file = 'data/screen/20200203.TXT'

### 2. Specifiy IRIS version

In [3]:
#Valid choices are : IRIS_1, IRIS_2, or IRIS_3
IRIS_VERSION = 'IRIS_2'

### 3. Processing

#### Read file and create calibration and measures dataframe

In [4]:
df_raw = pd.read_csv(iris_file, header=None, names=['date', 'time', 'height', 'voltage'])
df_raw['timestamp'] =  pd.to_datetime(df_raw['date'] + " " +  df_raw['time'])
df_raw.drop(['date', 'time'], axis=1, inplace=True)

calib_df  = df_raw[df_raw['height'].str.contains('Spectralon')].copy()
measures_df = df_raw[df_raw['height'].str.contains(' cm')].copy()

#### Compute reflectance voltage averages for spectralon and measures

In [5]:
calib_df.rename({'height': 'spectralon'}, inplace=True, axis=1)
calib_df['spectralon'] = calib_df['spectralon'].apply(lambda x : int(x.split(' ')[-1][:-1]))
calib_df = calib_df.groupby(['spectralon']).mean()

#### Create averaged voltages per heights

In [6]:
measures_df['height'] = measures_df.apply(lambda x : int(x['height'].split('cm')[0]), axis=1)
measures_df = measures_df.groupby(['height'], axis=0).mean()

#### Compute IRIS parameters

In [7]:
# Voltage to reflectance polynomial fit
polynom = iris.calibration_polynom_fit(calib_df.index.values, calib_df['voltage'].values)

In [8]:
# Compute reflectance
measures_df['reflectance'] = measures_df['voltage'].apply(lambda x : iris.voltage_to_reflectance(x, polynom))

In [9]:
# Compute SSA from reflectance
measures_df['ssa'] = measures_df['reflectance'].apply(lambda x : iris.reflectance_to_ssa(x, IRIS_VERSION))

In [10]:
# Compute optical radius from SSA
measures_df['optical radius'] = measures_df['ssa'].apply(lambda x : iris.ssa_to_optical_radius(x))

#### Processed data preview

In [14]:
measures_df.sort_index(ascending=False, inplace=True)
measures_df

,voltage,reflectance,ssa,optical radius
height,,,,
305,1.534000,51.35,55.17,0.059319
292,1.307000,40.71,30.34,0.107864
280,1.243000,38.46,26.84,0.121930
271,1.322333,41.29,31.32,0.104489
265,1.309333,40.79,30.48,0.107369
260,1.225333,37.88,26.01,0.125821
255,1.059333,33.14,20.09,0.162897
245,1.180333,36.49,24.11,0.135737
239,1.154333,35.73,23.14,0.141426


### 4. Write dataframe to a .csv file

In [15]:
measures_df.to_csv(iris_file.split('.')[0] + '_processed.csv')